# Building Custom Curves (Nelson-Siegel)

The *rateslib* curves objects are structured in such a way that it is
strightforward to create new objects useable throughout the library, provided they implement
the necessary abstract base objects.

## Nelson-Siegel Parametrization

This example will construct a parametric **Nelson-Siegel Curve**, whose continuously
compounded zero rate, $r$, at time, $T$, is given by the following
equation of **four** parameters:

$$
r(T) = \begin{bmatrix} \beta_0 & \beta_1 & \beta_2 \end{bmatrix} \begin{bmatrix} 1 \\ \lambda (1- e^{-T/ \lambda}) / T \\ \lambda (1- e^{-T/ \lambda})/ T - e^{-T/ \lambda} \end{bmatrix}
$$

This leads to the **discount factors** on that curve equaling:

$$
v(T) = e^{-T r(T)}
$$

## _BaseCurve ABCs

First we setup the skeletal structure of our custom curve. We will inherit from
[_BaseCurve][basecurve] and setup the necessary abstract base class (ABC) properties.

Some items we know in advance regarding our custom curve:

- it will return discount factors, defining its [_CurveType][curvetype],
- it has four parameters,
- we must define a ``start`` date (which makes its initial node) and ``end``.

A [_BaseCurve][basecurve] requires implementations of
[roll()][basecurve], [shift()][basecurve], [translate()][basecurve].
However, we can acquire these automatically by
inheriting the [_WithOperations][withops] class.

[basecurve]: ./api/rateslib.curves._BaseCurve.rst
[curvetype]: ./api/rateslib.curves._CurveType.rst
[withops]: ./api/rateslib.curves._WithOperations.rst

In [ ]:
from rateslib import *
import numpy as np

from rateslib.curves import _BaseCurve, _CurveType, _WithOperations, _CurveNodes

class NelsonSiegelCurve(_WithOperations, _BaseCurve):
    
    # ABC properties 
    
    _base_type = _CurveType.dfs
    _id = None
    _meta = None
    _nodes = None
    _ad = None
    _interpolator = None
    _n = 4
   
    def __init__(self, start, end, beta0, beta1, beta2, lambd):
        self._id = super()._id
        self._meta = super()._meta
        self._ad = 0
        self._nodes = _CurveNodes({start: 0.0, end: 0.0})
        self._params = (beta0, beta1, beta2, lambd)

    # ABC required methods
    
    def _set_ad_order(self, order):
        raise NotImplementedError()
   
    def __getitem__(self, date):
        raise NotImplementedError()

This curve can now be initialised without raising any errors relating to *Abstract Base Classes*.
However, it doesn't do much without implementing the ``__getitem__`` method.

In [ ]:
curve = NelsonSiegelCurve(dt(2000, 1, 1), dt(2010, 1, 1), 0.025, 0.0, 0.0, 0.5)

## The `__getitem__` method

This method will return discount factors. If the requested date is prior to the curve: return zero as usual. If the date is the
same as the initial node date: return one, else use continuously compounded rates to
derive a discount factor.

In [ ]:
class NelsonSiegelCurve(_WithOperations, _BaseCurve):
    _base_type = _CurveType.dfs
    _id = None
    _meta = None
    _nodes = None
    _ad = None
    _interpolator = None
    _n = 4
    
    def __init__(self, start, end, beta0, beta1, beta2, lambd):
        self._id = super()._id
        self._meta = super()._meta
        self._ad = 0
        self._nodes = _CurveNodes({start: 0.0, end: 0.0})
        self._params = (beta0, beta1, beta2, lambd)
    
    def _set_ad_order(self, order):
        raise NotImplementedError()
    
    def __getitem__(self, date):
        if date < self.nodes.initial:
            return 0.0
        elif date == self.nodes.initial:
            return 1.0
        b0, b1, b2, l0 = self._params
        T = dcf(self.nodes.initial, date, convention=self.meta.convention, calendar=self.meta.calendar)
        a1 = l0 * (1 - dual_exp(-T / l0)) / T
        a2 =  a1 - dual_exp(-T / l0)
        r = b0 + a1 * b1 + a2 * b2
        return dual_exp(-T * r)

Once this method is added to the class and the discount factors are available,
all of the provided methods are also available. This means the following are all
automatically functional:

In [ ]:
ns_curve = NelsonSiegelCurve(
    start=dt(2000, 1, 1), 
    end=dt(2010, 1, 1), 
    beta0=0.03, 
    beta1=-0.01, 
    beta2=0.01, 
    lambd=0.75
)
ns_curve.plot("1b", comparators=[ns_curve.shift(100), ns_curve.roll("6m")])

## Mutatbility and the ``Solver``

In order to allow this curve to be calibrated by a [Solver][solver],
we need to add some elements that allows the [Solver][solver] to interact
with it. We will also set the `NelsonSeigelCurve` to inherit
[_WithMutability][withmut].

Firstly, we can add the ``getter`` methods (NumPy is needed for this).
Make sure that ``_ini_solve = 0`` is added as a property to the class.


[solver]: ./api/rateslib.solver.Solver.rst
[withmut]: ./api/rateslib.curves._WithMutability.rst

In [ ]:
_ini_solve = 0

def _get_node_vector(self):
   return np.array(self._params)

def _get_node_vars(self):
   return tuple(f"{self._id}{i}" for i in range(self._ini_solve, self._n))

The ``setter`` method that the [Solver][solver] needs is slightly
more complicated. It requires state management, which we can easily add with an imported decorator. The additional
methods are shown below.

[solver]: ./api/rateslib.solver.Solver.rst

In [ ]:
from rateslib.curves import _WithMutability
from rateslib.mutability import _new_state_post
from rateslib.dual import set_order_convert
from rateslib.dual.utils import _dual_float

@_new_state_post
def _set_node_vector(self, vector, ad):
    if ad == 0:
        self._params = tuple(_dual_float(_) for _ in vector)
    elif ad == 1:
        self._params = tuple(
            Dual(_dual_float(_), [f"{self._id}{i}"], []) for i, _ in enumerate(vector)
        )
    else: # ad == 2
        self._params = tuple(
            Dual2(_dual_float(_), [f"{self._id}{i}"], [], []) for i, _ in enumerate(vector)
        )

def _set_ad_order(self, order):
    if self.ad == order:
        return None
    else:
        self._params = tuple(
            set_order_convert(_, order, [f"{self._id}{i}"]) for i, _ in enumerate(self._params)
        )

Adding these elements yields the final code class:

In [ ]:
class NelsonSiegelCurve(_WithMutability, _WithOperations, _BaseCurve):

    # ABC properties
    
    _ini_solve = 0
    _base_type = _CurveType.dfs
    _id = None
    _meta = None
    _nodes = None
    _ad = None
    _interpolator = None
    _n = 4
    
    def __init__(self, start, end, beta0, beta1, beta2, lamb):
        self._id = super()._id
        self._meta = super()._meta
        self._ad = 0
        self._nodes = _CurveNodes({start: 0.0, end: 0.0})
        self._params = (beta0, beta1, beta2, lamb)

    def __getitem__(self, date):
        if date < self.nodes.initial:
            return 0.0
        elif date == self.nodes.initial:
            return 1.0
        b0, b1, b2, l0 = self._params
        T = dcf(self.nodes.initial, date, convention=self.meta.convention, calendar=self.meta.calendar)
        a1 = l0 * (1 - dual_exp(-T / l0)) / T
        a2 =  a1 - dual_exp(-T / l0)
        r = b0 + a1 * b1 + a2 * b2
        return dual_exp(-T * r)

    # Solver mutability methods
    
    def _get_node_vector(self):
        return np.array(self._params)
    
    def _get_node_vars(self):
        return tuple(f"{self._id}{i}" for i in range(self._ini_solve, self._n))
    
    def _set_node_vector(self, vector, ad):
        if ad == 0:
            self._params = tuple(_dual_float(_) for _ in vector)
        elif ad == 1:
            self._params = tuple(
                 Dual(_dual_float(_), [f"{self._id}{i}"], []) for i, _ in enumerate(vector)
            )
        else: # ad == 2
            self._params = tuple(
                Dual2(_dual_float(_), [f"{self._id}{i}"], [], []) for i, _ in enumerate(vector)
            )
    
    def _set_ad_order(self, order):
        if self.ad == order:
            return None
        else:
            self._params = tuple(
                set_order_convert(_, order, [f"{self._id}{i}"]) for i, _ in enumerate(self._params)
            )

In [ ]:
ns_curve = NelsonSiegelCurve(dt(2000, 1, 1), dt(2010, 1, 1), 0.03, -0.01, 0.01, 0.75)
solver = Solver(
    curves=[ns_curve],
    instruments=[
        IRS(dt(2000, 1, 1), "2y", "A", curves=ns_curve),
        IRS(dt(2000, 1, 1), "4y", "A", curves=ns_curve),
        IRS(dt(2000, 1, 1), "7y", "A", curves=ns_curve),
    ],
    s=[2.45, 2.90, 2.66],
    id="NS",
    instrument_labels=["2y", "4y", "7y"],
)

In [ ]:
ns_curve.plot("1b")

## Risk

Since the [Solver][solver] has been invoked all typically delta and gamma methods can now also be used against this curve risk model

[solver]: ./api/rateslib.solver.Solver.rst

In [ ]:
portfolio = IRS(dt(2003, 7, 1), "4y", "A", notional=50e6, fixed_rate=4.5, curves=ns_curve)
portfolio.delta(solver=solver)

In [ ]:
portfolio.gamma(solver=solver)